## Unificar varios archivos csv en uno

In [ ]:
import os
import csv
from tkinter import Tk, filedialog

def detectar_encoding(archivo):
    # Intenta detectar la codificación común
    encodings = ['utf-8', 'latin-1', 'cp1252', 'iso-8859-1']
    for enc in encodings:
        try:
            with open(archivo, 'r', encoding=enc) as f:
                f.read(10000)  # Lee una muestra
                return enc
        except:
            continue
    return 'latin-1'  # Por defecto si no se detecta

def unificar_csv():
    root = Tk()
    root.withdraw()
    
    print("Selecciona los archivos CSV que deseas unificar:")
    archivos = filedialog.askopenfilenames(
        title="Seleccionar archivos CSV",
        filetypes=[("Archivos CSV", "*.csv"), ("Todos los archivos", "*.*")]
    )
    
    if not archivos:
        print("No se seleccionaron archivos. Saliendo...")
        return
    
    print("\nSelecciona la ubicación y nombre para el archivo unificado:")
    archivo_salida = filedialog.asksaveasfilename(
        title="Guardar archivo unificado como",
        defaultextension=".csv",
        filetypes=[("Archivos CSV", "*.csv"), ("Todos los archivos", "*.*")]
    )
    
    if not archivo_salida:
        print("No se seleccionó ubicación de salida. Saliendo...")
        return
    
    encabezados = None
    filas_totales = 0
    
    with open(archivo_salida, 'w', newline='', encoding='utf-8') as f_out:
        writer = None
        
        for archivo in archivos:
            try:
                encoding = detectar_encoding(archivo)
                print(f"Procesando {os.path.basename(archivo)} con encoding: {encoding}")
                
                with open(archivo, 'r', newline='', encoding=encoding) as f_in:
                    # Algunos CSVs usan punto y coma (;) en lugar de coma (,)
                    dialect = csv.Sniffer().sniff(f_in.read(1024))
                    f_in.seek(0)
                    
                    reader = csv.reader(f_in, dialect)
                    encabezados_actuales = next(reader)
                    
                    if encabezados is None:
                        encabezados = encabezados_actuales
                        writer = csv.writer(f_out)
                        writer.writerow(encabezados)
                    elif encabezados_actuales != encabezados:
                        print(f"Advertencia: Los encabezados en {os.path.basename(archivo)} no coinciden. Se omitirá.")
                        continue
                    
                    filas = 0
                    for row in reader:
                        writer.writerow(row)
                        filas += 1
                    
                    filas_totales += filas
                    print(f"Procesado: {os.path.basename(archivo)} - {filas} filas")
                    
            except Exception as e:
                print(f"Error al procesar {os.path.basename(archivo)}: {str(e)}")
    
    print(f"\n¡Unificación completada!")
    print(f"Archivo unificado guardado en: {archivo_salida}")
    print(f"Total de archivos procesados: {len([a for a in archivos if os.path.exists(a)])}")
    print(f"Total de filas en el archivo unificado: {filas_totales}")

if __name__ == "__main__":
    unificar_csv()

## Cargando el archivo .csv para limpiarlo

In [ ]:
from IPython.display import display
import ipywidgets as widgets
import pandas as pd
import unidecode

# Crear widget para subir archivo
uploader = widgets.FileUpload(
    accept='.csv',  # Aceptar solo archivos CSV
    multiple=False  # Permitir solo un archivo
)

display(uploader)  # Mostrar el widget de subida

# Esperar a que se suba el archivo
def procesar_archivo(change):
    # Obtener el nombre y contenido del archivo subido
    file_name = next(iter(uploader.value))
    content = uploader.value[file_name]['content']
    
    # Cargar el CSV desde el contenido en memoria
    df = pd.read_csv(io.BytesIO(content), encoding="utf-8", sep=",")
    
    # Resto de tu procesamiento...
    if "ship_name.1" in df.columns:
        df = df.drop(columns=["ship_name.1"])

    # Función para limpiar caracteres extraños
    def clean_text(text):
        if isinstance(text, str):
            text = unidecode.unidecode(text)
            text = text.strip()
            return text
        return text

    for col in df.columns:
        df[col] = df[col].astype(str).apply(clean_text)
    
    # Guardar CSV limpio
    df.to_csv("datos_maritimos_limpios.csv", index=False, encoding="utf-8")
    
    print("Procesamiento completado!")
    print(df.head())

# Observar cambios en el widget
uploader.observe(procesar_archivo, names='value')

ModuleNotFoundError: No module named 'pandas'